<a href="https://colab.research.google.com/github/RuoguLi0425/Robot-Learning/blob/main/Robot_Learning_Project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Important***

**Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/172081/pages/assignment-instructions) page on Courseworks2 to learn the workflow for completing this project.**

<ins> Different from project 1 and 2</ins>, apart from the link to your notebook, you are also required to submit 2 additional files namely the collected data `data.pkl` and your chosen model checkpoint `dynamic.pth`.

## Project Setup

In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp23_project4.git

Cloning into 'mecs6616_sp23_project4'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15 (delta 5), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 8.63 KiB | 2.88 MiB/s, done.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp23_project4/* /content/

In [ ]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 45.6 MB/s eta 0:00:00


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
from collections import defaultdict
import numpy as np

class MPC:

    def __init__(self):
        self.control_horizon = 10
        # Define other parameters here

        self.times = 40
        self.DeltaU  =    {1: [0.2],
                    2: [0.27, 0.4],
                    3: [0.1, 0.2, 0.2]}

        self.weights = {1: (1, 0.1),
                 2: (1.2, 0.1),
                 3: (1, 0.1)}

    def get_trajectory(self, state, action, dynamics):
        # Initialize the trajectory with the initial state
        traj = [state]

        traj_len = self.times

        # Check if the trajectory length is greater than 0, raise a ValueError otherwise
        if traj_len <= 0:
            raise ValueError("trajectory length (self.times) must be greater than 0")

        counter = 0

        while counter < traj_len:
            # Compute the new state by advancing the dynamics with the last state in the trajectory and the given action
            new_state = dynamics.advance(traj[-1], action)

            traj.append(np.copy(new_state))

            counter += 1

        return np.array(traj)


    def compute_cost(self, dynamics, trajectory, goal, index):
        total_cost = 0

        # Retrieve the weights A and B from the self.weights dictionary using the number of links in the dynamics object
        A, B = self.weights[dynamics.num_links]

        traj_len = len(trajectory)

        # Check if the trajectory has at least two elements, raise a ValueError otherwise
        if traj_len < 2:
            raise ValueError("trajectory must have at least two elements")

        traj_index = 1

        while traj_index < traj_len:
            # Get the current element (joint angles) from the trajectory
            sk = trajectory[traj_index]

            # Compute the forward kinematics for the given joint angles
            new_pos = dynamics.compute_fk(sk)

            # Calculate the distance cost between the target and the new position, and add it to the total cost
            total_cost += A * np.linalg.norm(goal - new_pos)

            # Compute the end-effector velocity for the given joint angles
            vel_ee = np.linalg.norm(dynamics.compute_vel_ee(sk))

            # Add the velocity cost to the total cost
            total_cost += B * vel_ee


            traj_index += 1

        return total_cost

    def compute_action(self, dynamics, state, goal, action):
        # Get the length of the action
        n = len(action)

        # Check if the action length is greater than 0, raise a ValueError otherwise
        if n <= 0:
            raise ValueError("action length must be greater than 0")

        traj = self.get_trajectory(state, action, dynamics)
        best_cost = self.compute_cost(dynamics, traj, goal, n)
        best_action = np.copy(action)

        improving = True
        delta_vals = self.DeltaU[dynamics.num_links]

        # Check if the length of delta_vals is the same as the action length, raise a ValueError otherwise
        if len(delta_vals) != n:
            raise ValueError("delta_values length must match action length")

        # Iterate until no improvement is found
        while improving:
            improving = False

            candidates = []
            for i in range(n):
                pos_candidate = np.copy(best_action)
                pos_candidate[i, 0] += delta_vals[i]
                candidates.append(pos_candidate)

                neg_candidate = np.copy(best_action)
                neg_candidate[i, 0] -= delta_vals[i]
                candidates.append(neg_candidate)

            # Evaluate candidates and update the best action if a better candidate is found
            for candidate in candidates:
                new_traj = self.get_trajectory(state, candidate, dynamics)
                new_cost = self.compute_cost(dynamics, new_traj, goal, n)
                if new_cost < best_cost:
                    best_cost = new_cost
                    improving = True
                    best_action = np.copy(candidate)

        return best_action


# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False


if gui:
  renderer = Renderer()
  time.sleep(0.5)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] =  0.76076509
goal[1, 0] =  -0.00775128
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)

# Initialize the action variable
action = np.zeros((arm.dynamics.get_action_dim(), 1))

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon == 0:
      state = arm.get_state()

      # Measuring distance and velocity of end effector
      pos_ee = dynamics_teacher.compute_fk(state)
      dist = np.linalg.norm(goal-pos_ee)
      vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
      print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

      action = controller.compute_action(arm.dynamics, state, goal, action)
      arm.set_action(action)


At timestep 0: Distance to goal: 2.132561484343264, Velocity of end effector: 7.715274834628325e-18
At timestep 10: Distance to goal: 1.938013756539036, Velocity of end effector: 2.7543815698658727
At timestep 20: Distance to goal: 1.4434193931239916, Velocity of end effector: 1.603007716428472
At timestep 30: Distance to goal: 0.9961014793878289, Velocity of end effector: 0.47560821510026885
At timestep 40: Distance to goal: 0.7118821444343756, Velocity of end effector: 0.09535283433973675
At timestep 50: Distance to goal: 0.5672856051859186, Velocity of end effector: 0.11984914011238927
At timestep 60: Distance to goal: 0.5270501984769096, Velocity of end effector: 0.007658179754528301
At timestep 70: Distance to goal: 0.5402318918633511, Velocity of end effector: 0.06767591480740884
At timestep 80: Distance to goal: 0.54796420922055, Velocity of end effector: 0.1840805744442144
At timestep 90: Distance to goal: 0.5057945186709515, Velocity of end effector: 1.6090161014889564
At time

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.40824467 -0.91287255], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.68202991 -0.73132428], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.71862797 -1.57873182], Final velocity: [0.0049502]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.1795744  -1.37519327], Final velocity: [0.10741453]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.23573887 -2.4138002 ], Final velocity: [0.00955623]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-2.10367746 -1.35075576], Final position: [-2.09461199 -1.35526754], Final velocity: [0.01281574]
score: 1.0/1.0
       
----------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):

	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step

	def forward(self, x):
		qddot = self.compute_qddot(x)
		state = x[:, :2*self.num_links]
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):

		# Your code goes here
		q = state[:, :self.num_links]
		qdot = state[:, self.num_links:]
		qdot_new = qdot + qddot * self.time_step
		q_new = q + qdot_new * self.time_step
		next_state = torch.cat((q_new, qdot_new), dim=1)
		return next_state

	def compute_qddot(self, x):
		pass

class Model2Link(Model):

		def __init__(self, time_step):
				super().__init__(2, time_step)
				self.fc1 = nn.Linear(6, 64)
				self.fc2 = nn.Linear(64, 128)
				self.fc3 = nn.Linear(128, 256)
				self.fc4 = nn.Linear(256, 2)

		def compute_qddot(self, x):
				x = F.relu(self.fc1(x))
				x = F.relu(self.fc2(x))
				x = F.relu(self.fc3(x))
				qddot = self.fc4(x)
				return qddot


### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  num_samples = 1000000
  state_dim = arm.dynamics.get_state_dim()
  action_dim = arm.dynamics.get_action_dim()

  X = np.zeros((num_samples, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_samples, arm.dynamics.get_state_dim()))
  for i in range(num_samples):
        # Randomly sample an action
        action = np.random.uniform(-2.5, 2.5, (action_dim, 1))
        arm.set_action(action)

        # Get the current state before applying the action
        state = arm.get_state()
        x = np.concatenate((state, action), axis=0).reshape(-1)

        # Apply the action and get the next state
        arm.advance()
        next_state = arm.get_state()

        # Save the state-action pair and the next state
        X[i] = x
        Y[i] = next_state.reshape(-1)

        # Reset the arm every 100 steps
        if i % 100 == 0:
            arm.reset()
  # ---

  return X, Y


In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))

In [ ]:
print(X.shape,Y.shape)

(1000000, 6) (1000000, 4)


### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

LearningRate=0.01

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(model, train_loader):
  model.train()
  device = torch.device("cpu")
  optimizer = torch.optim.Adam(model.parameters(), lr=LearningRate)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=4, verbose=True, min_lr=1e-8)
  # ---
  # Your code goes here
  train_loss = 0
  criterion = nn.MSELoss()

  for train_X, train_Y in train_loader:
      train_X, train_Y = train_X.to(device), train_Y.to(device)

      optimizer.zero_grad()

      output = model(train_X)
      loss = criterion(output, train_Y)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

  scheduler.step(train_loss)

  return train_loss / len(train_loader)
  # ---
  pass

def test(model, test_loader):
  model.eval()
  device = torch.device("cpu")
  # --
  criterion = nn.MSELoss()
  test_loss = 0

  for test_X, test_Y in test_loader:
      test_X, test_Y = test_X.to(device), test_Y.to(device)
      outputs = model(test_X.float())
      loss = criterion(outputs.float(), test_Y.float())
      test_loss += loss.item()

  test_loss = test_loss / len(test_loader)
  print ('test loss: ', test_loss)
  # --
  return test_loss

def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path
  device = torch.device("cpu")
  model = Model2Link(0.01)
  model.to(device)

  datafile = 'dataset/data.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  batch_size = 500
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=LearningRate)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=4, verbose=True, min_lr=1e-8)

  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  model_path = None
  best_test_loss = float('inf')
  epochs = 50

  for epoch in range(1, 1 + epochs):
      train_loss = train_one_epoch(model, train_loader)
      test_loss = test(model, test_loader)
      scheduler.step(test_loss)

      if test_loss < best_test_loss:
          best_test_loss = test_loss
          best_epoch = epoch

          model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
          if not os.path.exists(os.path.join(model_dir, model_folder_name)):
              os.makedirs(os.path.join(model_dir, model_folder_name))
          torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))
          model_path = os.path.join(model_dir, model_folder_name, 'dynamics.pth')

      print(f'train_loss: {train_loss:.8f}, test_loss: {test_loss:.8f}, Epoch: {epoch}')
      print(f"The top-performing model was saved at: {model_path}")
      print(f"**********The best:{best_epoch} Now:{epoch}**********")

  return model_path


In [ ]:
model_path = train_forward_model()

test loss:  0.00047908981519867666
train_loss: 0.00888986, test_loss: 0.00047909, Epoch: 1
The top-performing model was saved at: models/2023-04-16_22-08-22/epoch_0001_loss_0.00047909/dynamics.pth
**********The best:1 Now:1**********
test loss:  0.0001773449731990695
train_loss: 0.00111288, test_loss: 0.00017734, Epoch: 2
The top-performing model was saved at: models/2023-04-16_22-08-22/epoch_0002_loss_0.00017734/dynamics.pth
**********The best:2 Now:2**********
test loss:  0.00013731221197303966
train_loss: 0.00065814, test_loss: 0.00013731, Epoch: 3
The top-performing model was saved at: models/2023-04-16_22-08-22/epoch_0003_loss_0.00013731/dynamics.pth
**********The best:3 Now:3**********
test loss:  0.0016036704886937514
train_loss: 0.00040066, test_loss: 0.00160367, Epoch: 4
The top-performing model was saved at: models/2023-04-16_22-08-22/epoch_0003_loss_0.00013731/dynamics.pth
**********The best:3 Now:4**********
test loss:  0.00011924579708647797
train_loss: 0.00040236, test_lo

### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase
import torch

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # Initialize the model loading the saved model from provided model_path
        try:
            # Create an instance of the neural network model
            self.model = Model2Link(time_step).to(device)
            # Load the saved state dict for the model
            model_state = torch.load(model_path, map_location=device)
            # Load the state dict into the model and set it to evaluation mode
            self.model.load_state_dict(model_state)
            np.random.seed(425)
            self.model.eval()
            # Mark the model as loaded
            self.model_loaded = True
        except:
            raise Exception("Error initializing model. Please check model_path, num_links, time_step, and device parameters.")

    def dynamics_step(self, state, action, dt):
        # Check if the model has been loaded
        if not self.model_loaded:
            raise Exception("Model not initialized. Please call init_model() before using dynamics_step().")
        # Combine the current state and action into a numpy array
        state_action = np.concatenate((state, action), axis=0)
        # Convert the state-action numpy array to a PyTorch tensor
        state_action_tensor = torch.from_numpy(state_action)
        # Convert the data type to float and move the tensor to the specified device
        state_action_tensor = state_action_tensor.float().to(device)
        # Reshape the tensor to have a batch size of 1 and a single dimension for the data
        state_action_tensor = state_action_tensor.view(1, -1)
        # Use the model to predict the new state
        with torch.no_grad():
            new_state_tensor = self.model(state_action_tensor)
        # Convert the new state tensor to a numpy array and reshape it
        new_state = new_state_tensor.cpu().numpy().reshape(-1, 1)
        # Return the new state
        return new_state

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)


# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
# model_path = None
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 1.5
goal[1, 0] = -0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
# model_path = 'dynamics.pth'
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Part2: EVALUATING CONTROLLER + LEARNED DYNAMICS
-----------------------------------------------
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [-0.98771631 -1.1159843 ], Final position: [-1.08122845 -1.13999432], Final velocity: [0.18756613]
score: 0.5/0.5
Test  2
Success! :)
 Goal: [-0.59945788 -1.48799136], Final position: [-0.56083737 -1.56156227], Final velocity: [0.05004236]
score: 0.5/0.5
Test  3
Success! :)
 Goal: [ 0.84958949 -0.81301908], Final position: [ 0.84579973 -0.82343866], Final velocity: [0.03486416]
score: 0.5/0.5
Test  4
Success! :)
 Goal: [ 0.1890371 -0.9494469], Final position: [ 0.28459396 -0.9822649 ], Final velocity: [0.04185975]
score: 0.5/0.5
Test  5
Fail :(
 Goal: [ 0.76076509 -0.00775128], Final position: [-0.06642668 -0.04843603], Final velocity: [0.29507383]
score: 0/0.5
Test  6
Success! :)
 Goal: [-0.25583319 -0.26144673], Final position: [-0.30042445 -0.12801912], Final velocity: [0.0673403]
score: 0.5/0.5
Test  7
Success! :)
 Goal: [-0.44661901 -1.73929781], 

Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy